In [1]:
import os
cwd = os.getcwd()
data_dir = cwd + '/dataset'
os.listdir(data_dir)

['prev_check2_clean.csv',
 'application_test.csv',
 '.DS_Store',
 'HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'train_set.csv',
 'test_set.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'bureau.csv',
 'bu_clean.csv',
 'prev_check1_clean.csv',
 'previous_application.csv',
 'HomeCredit_columns_description.xls',
 'bureau_balance.csv',
 'sample_submission.csv']

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns

import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV

%matplotlib inline
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

In [3]:
train_set = pd.read_csv(data_dir + "/train_set.csv",index_col='SK_ID_CURR')
test_set = pd.read_csv(data_dir + "/test_set.csv",index_col='SK_ID_CURR')

In [4]:
y_train = train_set["TARGET"]
x_train = train_set.drop(["TARGET"],axis = 1)
x_test = test_set

In [5]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=10)
#train_set.info()
#type(y_train[0])

In [38]:
print('Start training...')
gbm = lgb.LGBMRegressor(objective='binary',
                        max_depth=8,
                        #subsample=0.9,
                        #colsample_bytree=0.8,
                        #reg_alpha=0.1,
                        #reg_lambda=0.1,
                        #min_split_gain=0.01,
                        #min_child_weight=2,
                        #num_leaves=50,
                        #learning_rate=0.03,
                        n_estimators=2000,
                        #min_data_in_leaf=40,
                        is_ubalance=True,
                        #feature_fraction=0.8,
                        verbose=-1)

Start training...


In [ ]:
param_grid = {
    'learning_rate':[0.01,0.05,0.1],
    'num_leaves':[20, 30, 40, 50, 60],   
}
gbm_search = GridSearchCV(gbm, param_grid)
gbm_search.fit(X_train,Y_train,
               eval_set=[(X_valid,Y_valid)],
               eval_metric='auc',
               early_stopping_rounds=300,
               verbose=100)

Training until validation scores don't improve for 300 rounds.
[100]	valid_0's auc: 0.708724
[200]	valid_0's auc: 0.717886
[300]	valid_0's auc: 0.726019
[400]	valid_0's auc: 0.734302
[500]	valid_0's auc: 0.739555
[600]	valid_0's auc: 0.743333
[700]	valid_0's auc: 0.745701
[800]	valid_0's auc: 0.747298
[900]	valid_0's auc: 0.748626
[1000]	valid_0's auc: 0.749456
[1100]	valid_0's auc: 0.750037
[1200]	valid_0's auc: 0.75049
[1300]	valid_0's auc: 0.75083
[1400]	valid_0's auc: 0.751126
[1500]	valid_0's auc: 0.751375
[1600]	valid_0's auc: 0.751556
[1700]	valid_0's auc: 0.751659
[1800]	valid_0's auc: 0.751747
[1900]	valid_0's auc: 0.751837
[2000]	valid_0's auc: 0.751869
Did not meet early stopping. Best iteration is:
[1990]	valid_0's auc: 0.75189
Training until validation scores don't improve for 300 rounds.
[100]	valid_0's auc: 0.708059
[200]	valid_0's auc: 0.715858
[300]	valid_0's auc: 0.724631
[400]	valid_0's auc: 0.732596
[500]	valid_0's auc: 0.738682
[600]	valid_0's auc: 0.742295
[700]	v

In [33]:
gbm.fit(X_train,Y_train,
        eval_set=[(X_train,Y_train),(X_valid,Y_valid)],
        eval_metric='auc',
        verbose=100,
        )

Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.758017	valid_1's auc: 0.732217
[200]	training's auc: 0.786485	valid_1's auc: 0.747745
[300]	training's auc: 0.805308	valid_1's auc: 0.752886
[400]	training's auc: 0.819702	valid_1's auc: 0.75501
[500]	training's auc: 0.833216	valid_1's auc: 0.756204
[600]	training's auc: 0.844939	valid_1's auc: 0.75673
[700]	training's auc: 0.855696	valid_1's auc: 0.756689
[800]	training's auc: 0.866025	valid_1's auc: 0.75699
[900]	training's auc: 0.875015	valid_1's auc: 0.757155
[1000]	training's auc: 0.883589	valid_1's auc: 0.756897
[1100]	training's auc: 0.891481	valid_1's auc: 0.756933
Early stopping, best iteration is:
[856]	training's auc: 0.87098	valid_1's auc: 0.757235


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       is_ubalance=True, learning_rate=0.03, max_depth=8,
       min_child_samples=20, min_child_weight=2, min_split_gain=0.0,
       n_estimators=10000, n_jobs=-1, num_leaves=50, objective='binary',
       random_state=None, reg_alpha=0.1, reg_lambda=0.1, silent=True,
       subsample=0.9, subsample_for_bin=200000, subsample_freq=1,
       verbose=-1)

In [8]:
trace = go.Scatter(
    y = gbm.feature_importances_,
    x = x_train.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        color = gbm.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = x_train.columns.values,
    name = 'LGBM'
)
layout = go.Layout(
        autosize=True,
        title =  'Feature Importances',
        hovermode = 'closest',
        yaxis = dict(
            title = 'Feature Importance',
            ticklen = 5,
            gridwidth = 2
        ),
        showlegend = False
    )
fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig,filename='scatter')

In [9]:
y_pred = gbm.predict(x_test,num_iteration=gbm.best_iteration_)

In [10]:
y_pred[y_pred<0] = 0

In [11]:
result = pd.DataFrame({"SK_ID_CURR":np.array(test_set.index),
                       "TARGET":y_pred
                      })

In [12]:
result.to_csv('submission.csv',index=False)

In [13]:
train_test_split?

In [14]:
auc?

In [15]:
y_pred

array([0.01308556, 0.05365547, 0.01996055, ..., 0.02566437, 0.0409772 ,
       0.28155502])

In [16]:
(y_pred<0).any()

False